<a href="https://colab.research.google.com/github/dbamman/nlp23/blob/master/AP/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT for binary or multiclass document classification using the [CLS] token as the document representation; trains a model (on `train.txt`), uses `dev.txt` for early stopping, and evaluates performance on `test.txt`.  Reports test accuracy with 95% confidence intervals.

Before executing this notebook on Colab, make sure you're running on cuda (`Runtime > Change runtime type > GPU`) to make use of GPU speedups.

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertModel, BertTokenizer
import nltk
import torch
import torch.nn as nn
import numpy as np
import random
from scipy.stats import norm
import math

In [ ]:
# If you have your folder of data on your Google drive account, you can connect that here
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change this to the directory with your data
directory="/content/drive/MyDrive/ap_data/"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on {}".format(device))

Running on cuda


In [ ]:
def read_labels(filename):
    labels={}
    with open(filename) as file:
        for line in file:
            cols = line.split("\t")
            label = cols[2]
            if label not in labels:
                labels[label]=len(labels)
    return labels

In [ ]:
def read_data(filename, labels, max_data_points=1000):

    data = []
    data_labels = []
    with open(filename) as file:
        for line in file:
            cols = line.split("\t")
            label = cols[2]
            text = cols[3]

            data.append(text)
            data_labels.append(labels[label])


    # shuffle the data
    tmp = list(zip(data, data_labels))
    random.shuffle(tmp)
    data, data_labels = zip(*tmp)

    if max_data_points is None:
        return data, data_labels

    return data[:max_data_points], data_labels[:max_data_points]

In [ ]:
labels=read_labels("%s/train.txt" % directory)

In [ ]:
labels=read_labels('adjudicated.txt')

In [ ]:
labels

{'Adolescent': 0, 'Adult': 1, 'Child': 2, 'Young Adult': 3}

In [ ]:
total = read_data('adjudicated.txt', labels, None)

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['text'] = total[0]
df['label'] = total[1]

In [ ]:
from sklearn.model_selection import train_test_split
train_x, X_temp, train_y, y_temp = train_test_split(list(df['text']), list(df['label']), test_size=0.4, random_state=42)  # 60% for training
dev_x, test_x, dev_y, test_y = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Splitting the 40% into two 20% pieces

In [ ]:
train_x, train_y=read_data("%s/train.txt" % directory, labels, max_data_points=None)

In [ ]:
dev_x, dev_y=read_data("%s/dev.txt" % directory, labels, max_data_points=None)

In [ ]:
test_x, test_y=read_data("%s/test.txt" % directory, labels, max_data_points=None)

In [ ]:
def evaluate(model, x, y):
    model.eval()
    corr = 0.
    total = 0.
    with torch.no_grad():
        for x, y in zip(x, y):
            y_preds=model.forward(x)
            for idx, y_pred in enumerate(y_preds):
                prediction=torch.argmax(y_pred)
                if prediction == y[idx]:
                    corr += 1.
                total+=1
    return corr/total, total

In [ ]:
def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer, BertConfig
import random

class BERTClassifier(nn.Module):

    def __init__(self, bert_model_name, params):
        super(BERTClassifier, self).__init__()
        self.model_name = bert_model_name
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name, do_lower_case=params.get("doLowerCase", True))

        # Configure BERT model with customized settings
        self.config = BertConfig.from_pretrained(self.model_name,
                                                 output_hidden_states=True,
                                                 output_attentions=params.get("output_attentions", False))
        self.bert = BertModel.from_pretrained(self.model_name, config=self.config)

        self.dropout = nn.Dropout(params.get("dropout_rate", 0.1))
        self.num_labels = params["label_length"]
        self.fc = nn.Linear(params["embedding_size"], self.num_labels)

    def get_batches(self, all_x, all_y, batch_size=32, max_toks=510):
        # Shuffle the dataset to ensure random batch distribution
        zipped_data = list(zip(all_x, all_y))
        random.shuffle(zipped_data)
        all_x, all_y = zip(*zipped_data)

        batches_x, batches_y = [], []
        device = "cuda" if torch.cuda.is_available() else "cpu"

        for i in range(0, len(all_x), batch_size):
            x = all_x[i:i+batch_size]
            batch_x = self.tokenizer(x, padding=True, truncation=True, return_tensors="pt", max_length=max_toks)
            batch_y = all_y[i:i+batch_size]

            batches_x.append(batch_x.to(device))
            batches_y.append(torch.LongTensor(batch_y).to(device))

        return batches_x, batches_y

    def forward(self, batch_x):
        bert_output = self.bert(input_ids=batch_x["input_ids"],
                                attention_mask=batch_x["attention_mask"],
                                token_type_ids=batch_x["token_type_ids"])

        bert_hidden_states = bert_output.hidden_states
        out = bert_hidden_states[-1][:,0,:]
        out = self.dropout(out)
        out = self.fc(out)
        return out.squeeze()

def train(bert_model_name, model_filename, train_x, train_y, dev_x, dev_y, labels, embedding_size=768, doLowerCase=None, verbose=True):
    params = {
        "label_length": len(labels),
        "doLowerCase": doLowerCase,
        "embedding_size": embedding_size,
        "dropout_rate": 0.1,
        "output_attentions": False
    }
    bert_model = BERTClassifier(bert_model_name, params)
    #device = "cuda" if torch.cuda.is_available() else "cpu"
    bert_model.to(device)

    optimizer = torch.optim.Adam(bert_model.parameters(), lr=1e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    cross_entropy = nn.CrossEntropyLoss()

    batch_x, batch_y = bert_model.get_batches(train_x, train_y)
    dev_batch_x, dev_batch_y = bert_model.get_batches(dev_x, dev_y)

    num_epochs = 30
    best_dev_acc = 0.0
    patience = 5
    best_epoch = 0

    for epoch in range(num_epochs):
        bert_model.train()

        for x, y in zip(batch_x, batch_y):
            y_pred = bert_model(x)
            loss = cross_entropy(y_pred.view(-1, bert_model.num_labels), y.view(-1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        dev_accuracy, _ = evaluate(bert_model, dev_batch_x, dev_batch_y)  # ensure evaluate function returns accuracy and loss
        if verbose:
            print("Epoch %s, dev accuracy: %.3f" % (epoch, dev_accuracy))
        if dev_accuracy > best_dev_acc:
            torch.save(bert_model.state_dict(), model_filename)
            best_dev_acc = dev_accuracy
            best_epoch = epoch

        if epoch - best_epoch > patience:
            if verbose:
                print("No improvement in dev accuracy over %s epochs; stopping training" % patience)
            break

    bert_model.load_state_dict(torch.load(model_filename))
    if verbose:
        print("\nBest Performing Model achieves dev accuracy of : %.3f" % (best_dev_acc))
    return bert_model

In [ ]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from transformers import BertModel, BertTokenizer, BertConfig


In [ ]:
# small BERT -- can run on laptop
bert_model_name="google/bert_uncased_L-2_H-128_A-2"
model_filename="mybert.model"
embedding_size=128
doLowerCase=True

# bert-base -- slow on laptop; better on Colab
#bert_model_name="bert-base-cased"
#model_filename="mybert.model"
#embedding_size=768
#doLowerCase=False

model=train(bert_model_name, model_filename, train_x, train_y, dev_x, dev_y, labels, embedding_size=embedding_size, doLowerCase=doLowerCase)

Epoch 0, dev accuracy: 0.320
Epoch 1, dev accuracy: 0.310
Epoch 2, dev accuracy: 0.330
Epoch 3, dev accuracy: 0.350
Epoch 4, dev accuracy: 0.370
Epoch 5, dev accuracy: 0.370
Epoch 6, dev accuracy: 0.390
Epoch 7, dev accuracy: 0.400
Epoch 8, dev accuracy: 0.390
Epoch 9, dev accuracy: 0.390
Epoch 10, dev accuracy: 0.380
Epoch 11, dev accuracy: 0.380
Epoch 12, dev accuracy: 0.390
Epoch 13, dev accuracy: 0.390
No improvement in dev accuracy over 5 epochs; stopping training

Best Performing Model achieves dev accuracy of : 0.400


In [ ]:
test_batch_x, test_batch_y = model.get_batches(test_x, test_y)
accuracy, test_n=evaluate(model, test_batch_x, test_batch_y)

lower, upper=confidence_intervals(accuracy, test_n, .95)
print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

Test accuracy for best dev model: 0.440, 95% CIs: [0.343 0.537]



In [ ]:
labels

{'Adolescent': 0, 'Adult': 1, 'Child': 2, 'Young Adult': 3}